In [ ]:
# investigation into vocabulary expansion the Learning Model (LM) described in https://arxiv.org/pdf/1806.10306.pdf [1]

In [ ]:
from lang.corporal import make_corpus, stringify_corpus

In [ ]:
np.random.seed(1)

In [ ]:
def one_encode(indices, vocab_size):
    encoding = []
    for i in indices:
        enc = np.zeros((vocab_size))
        enc[i] = 1
        encoding.append(enc)
    return np.array(encoding)

def sample(sess, inp, prob, seed_ix, n):
    # Initialize first word of sample ('seed') as one-hot encoded vector.
    x = np.zeros(inp.shape())
    x[seed_ix] = 1
    ixes = [seed_ix]

    for _ in range(n):
        inp.assign(x.T)
        sess.update_target([prob])
        p = prob.get() 
        p /= p.sum() # normalize

        # Choose next char according to the distribution
        ix = np.random.choice(range(p.shape[0]), p=p.ravel())
        x = np.zeros(inp.shape())
        x[ix] = 1
        ixes.append(ix)

    return ixes

def encoded_loss(encoded_expect, encoded_result):
    return tc.reduce_sum(-tc.log(tc.reduce_sum(encoded_result * encoded_expect, 0, 1)))

In [ ]:
sess = tc.Session()

In [ ]:
def multi_replace(txt, src_set, target):
    for c in src_set:
        txt = txt.replace(c, target)
    return txt
    
f = open('books.txt', 'r')
txt = f.read()
txt = txt.lower()
txt = multi_replace(txt, '.?,!()', ' \sep ')
txt = multi_replace(txt, ':;', ' \col ')
txt = multi_replace(txt, '-', ' \dsh ')
txt = txt.\
    replace('i\'m', 'i am').\
    replace('can\'t', 'can not').\
    replace('n\'t', ' not').\
    replace('\'re', ' are').\
    replace('\'ve', ' have').\
    replace('\'ll', ' will').\
    replace('\'d', ' would').\
    replace('\'s', '')
txt = multi_replace(txt, '"', '')
words = txt.split()
delims = ['\sep', '\col', '\dsh']
vocab = set(words)
vocab = [v for v in vocab if len(v) > 0 and (v.isalpha() or v in delims)]
data = []
for word in words:
    if word in vocab and (len(data) == 0 or word != data[-1]):
        data.append(data)
print(len(vocab))
print(len(data))

In [ ]:
data, order, vocab = make_corpus('lang/grammar.json', 'lang/vocabulary3.txt', 25, 10000)
# flatten data
data = [e for line in data for e in line]
data_size, vocab_size = len(data), len(order)
print('data has %d characters, %d unique.' % (data_size, vocab_size))

In [ ]:
seq_length = 25
learning_rate = 1e-1

gru = tc.layer.gru(vocab_size, vocab_size, seq_length,
    weight_init=tc.unif_xavier_init(1),
    bias_init=tc.zero_init())
dense = tc.layer.dense([vocab_size], [vocab_size],
    weight_init=tc.unif_xavier_init(1),
    bias_init=tc.zero_init())
model = tc.layer.link([
    gru,
    dense,
    tc.layer.bind(lambda x: tc.softmax(x, 0, 1)),
])

data_gru = gru.get_storage()[0].get()
data_dense = dense.get_storage()[0].get()

sample_invar = tc.EVariable([1, vocab_size], 0)
sample_prob = tc.slice(model.connect(sample_invar), 0, 1, 1)

invar = tc.EVariable([seq_length, vocab_size], 0)
exout = tc.EVariable([seq_length, vocab_size], 0)

train_err = tc.sgd_train(model, invar, exout,
    update=tc.get_adagrad(learning_rate=learning_rate, epsilon=1e-8),
    err_func=encoded_loss)
sess.track([sample_prob, train_err])

smooth_loss = -np.log(1.0/vocab_size)*seq_length
p = 0
print_interval = 100
for i in range(15000):
    # Reset memory if appropriate
    if p + seq_length + 1 >= len(data) or i == 0:
        p = 0

    # Get input and target sequence
    inputs = [ix for ix in data[p:p+seq_length]]
    encoded_inp = one_encode(inputs, vocab_size)
    encoded_out = one_encode([ix for ix in data[p+1:p+seq_length+1]], vocab_size)

    # Occasionally sample from oldModel and print result
    if i % print_interval == 0:
        sample_ix = sample(sess, sample_invar, sample_prob, inputs[0], 1000)
        print('----\n%s\n----' % (' '.join(order[ix] for ix in sample_ix)))

    # Get gradients for current oldModel based on input and target sequences
    invar.assign(encoded_inp)
    exout.assign(encoded_out)
    sess.update_target([train_err])
    loss = train_err.get()

    if i % print_interval == 0:
        print('iter %d, loss: %f, smooth loss: %f' % (i, loss, smooth_loss))

    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    i += 1
    
first_sample = 'data/sample1.onnx'
if tc.save_layers_file(first_sample, [model]):
    print('successfully saved to {}'.format(first_sample))